In [918]:
import pandas as pd
import sklearn
import time
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import XGBClassifier
from numpy import reshape
from sklearn import metrics
from collections import Counter 

In [919]:
data = pd.read_csv('train_dec08_task3.csv')

In [920]:
# split train and test data
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

In [921]:
# split features and labels
X_train = train_data.iloc[:, :14]
y_train = train_data.iloc[:, 14]

X_test = test_data.iloc[:, :14]
y_test = test_data.iloc[:, 14]

pred_data = pd.read_csv('test_dec08_task3_only_features.csv')

In [922]:
# check the imbalance of the data
Counter(y_train)

Counter({'A': 790, 'C': 2003, 'B': 1594, 'D': 813})

In [923]:
# use SMOTE to balance the data
# deprecated: SMOTE + TomekLinks is not better than SMOTE
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import TomekLinks
X_train, y_train = SMOTE(random_state=42).fit_resample(X_train, y_train)
# X_res, y_res = TomekLinks().fit_resample(X_re, y_re)

# check the imbalance of the data again
print(f"Training target statistics: {Counter(y_res)}")
print(f"Testing target statistics: {Counter(y_test)}")

Training target statistics: Counter({1: 1999, 3: 1999, 2: 1999, 0: 1999})
Testing target statistics: Counter({'C': 497, 'B': 406, 'A': 210, 'D': 187})


In [924]:
# #correlation matrix
# corrmat = X_train.corr()
# f, ax = plt.subplots(figsize=(12, 9))
# sns.heatmap(corrmat, vmax=.8, square=True)

In [925]:
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest

# 選擇要保留的特徵數
# select_k = 8
select_k = 5

selection = SelectKBest(mutual_info_classif, k=select_k).fit(X_train, y_train)

# 顯示保留的欄位
features = X_train.columns[selection.get_support()]
print(f"features: {features}")
X_train = selection.fit_transform(X_train, y_train)
X_test = selection.transform(X_test)


features: Index(['feature2', 'feature3', 'feature4', 'feature5', 'feature6', 'feature7',
       'feature11'],
      dtype='object')


In [926]:
# tranform the labels to numbers to fit XGBoost
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.fit_transform(y_test)

In [927]:
# build XGBClassifier model
xgboostModel = XGBClassifier(n_estimators=100, learning_rate= 0.36)

# train data with train_data
# xgboostModel.fit(X_train, y_train)
xgboostModel.fit(X_train, y_train)

# predict with the test_data
predicted = xgboostModel.predict(X_test)

In [928]:
# print the accuracy
# print('訓練集: ',xgboostModel.score(X_train,y_train))
print('訓練集: ',xgboostModel.score(X_train,y_train))
print('測試集: ',xgboostModel.score(X_test,y_test))

訓練集:  0.9921367948077883
測試集:  0.7184615384615385


In [929]:
# predict the real given data
pred_data = selection.transform(pred_data)
answer = xgboostModel.predict(pred_data)
answer = le.inverse_transform(answer)

In [930]:
da = {"Id": [x for x in range(1, len(answer)+1)], "Category": answer}
df = pd.DataFrame (da)

In [931]:
df.to_csv("output.csv", index=False)